<a href="https://colab.research.google.com/github/Eliascc5/English_proficiency_prediction_NLP/blob/main/projet_IAS_partie2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [143]:
#As in the first stage -> we load our dataset from Drive 
#Dataset: NICT_JLE_4.1
#Reference: https://alaginrc.nict.go.jp/nict_jle/index_E.html

from google.colab import drive
drive.mount("/content/gdrive")

#----------------------------------


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [144]:
import os

path_test= r'/content/gdrive/MyDrive/NICT_JLE_4.1/Output/'
os.chdir(path_test)

list_vocabulary=[]
y_output=[]

for file in os.listdir():
  if file.endswith(".txt"):

    file_path = f"{path_test}/{file}"

    with open(file_path, mode='r',encoding="utf8",errors='ignore') as f:

      score = f.readline()
      content = f.read()

      list_vocabulary.append(content)

      y_output.append(int(score))

      f.close

######################################

In [ ]:
text = list_vocabulary

# create the transform
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(text)
# summarize
print(vectorizer.vocabulary_)
# encode document
vector = vectorizer.transform(text)
# summarize encoded vector

#print(vector.shape)
#print(type(vector))
#print(vector.toarray())

extraemos los resultados 

ACA ARRANCAMOS CON LA RED !!!!!!!!!!!!!!!

In [146]:
from keras import models
from keras import layers
from keras.utils import np_utils
import tensorflow as tf
import numpy as np
#######################################
n=1281  #candidatos
m=15157 #son las palabras sin repeticion del vocabulario

array_output = np.array(y_output)

array_output = np_utils.to_categorical(array_output)

array_output = array_output[:,1:10]


########################################

model = models.Sequential()
model.add(layers.Dense(7000, activation='sigmoid', input_shape=(m,)))
model.add(layers.Dense(200, activation='sigmoid'))
model.add(layers.Dense(50, activation='sigmoid'))
model.add(layers.Dense(9, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


array=vector.toarray()

x_val_array=array[:round(0.2*n)]   # 20 porciento de los candidatos

y_val = array_output[:round(0.2*n)]

partial_x_train_array = array[round(0.2*n):]

partial_y_train_array = array_output[round(0.2*n):]

history = model.fit(partial_x_train_array,
                    partial_y_train_array,
                    epochs=40,
                    batch_size=300,
                    validation_data=(x_val_array, y_val))

#plot the results :
history_dict = history.history

import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


Epoch 1/40
1/4 [======>.......................] - ETA: 13s - loss: 3.0324 - accuracy: 0.0233

KeyboardInterrupt: ignored